# Homework 9
## Damian Franco
## CS-575

This program has two parts to the code. The first is setting up a matrix A and finding the iteration matrix and properties of the matrix to determine if the matrix will converge using Gauss-Seidel's algorithm. The next section (Question 3) is an implementation of Jacobi's and Gauss-Seidels algorithms to solve a linear system Ax = b with some examples and tables of my findings.

In [86]:
# Importing the required modules
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import time
from prettytable import PrettyTable
import scipy
import scipy.linalg 
from pprint import pprint
%matplotlib inline
import math

In [87]:
# Saves the current plot to desktop since working in Google Colab
from google.colab import files
#plt.savefig("my_plot.png", bbox_inches='tight', dpi=300)
#files.download("my_plot.png")

# Question 2

In [278]:
# Set up matrix A
A_GS = np.array([[1, -1, 0], [-1, 2, -1], [0, -1, 2]])
A_GS

array([[ 1, -1,  0],
       [-1,  2, -1],
       [ 0, -1,  2]])

## Check if A is invertible

In [281]:
# Compute the determinant of A
det_A_GS = np.linalg.det(A_GS)

if det_A_GS != 0:
    print("A is invertible")
else:
    print("A is not invertible")

A is invertible


## Find the iteration matrix M

In [284]:
# Compute the iteration matrix
D = np.diag(np.diag(A_GS)) 
L = np.tril(A_GS, -1)
U = np.triu(A_GS, 1)
M_GS = np.linalg.inv(D - L) @ U
print(M_GS)

[[ 0.   -1.    0.  ]
 [ 0.    0.5  -0.5 ]
 [ 0.   -0.25  0.25]]


## Find spectral radius p(M) of M 

In [285]:
# Compute the eigenvalues and p(M)
eigVals = np.linalg.eigvals(M_GS)
spectral_radius = np.max(np.abs(eigVals))
print("Spectral radius =", spectral_radius)

Spectral radius = 0.75


## Find norms of M

In [286]:
# 1-norm
one_norm = np.linalg.norm(M_GS, ord=1)
print("1-norm =", one_norm)

1-norm = 1.75


In [287]:
# 2-norm
two_norm = np.linalg.norm(M_GS, ord=2)
print("2-norm =", two_norm)

2-norm = 1.1841130945790346


In [288]:
# Infinity norm
inf_norm = np.linalg.norm(M_GS, ord=np.inf)
print("Infinity norm =", inf_norm)

Infinity norm = 1.0


# Question 3

## Form sparse linear system

In [88]:
f = lambda x: 4 * np.pi**2 * np.sin(2 * np.pi * x)

In [111]:
def form_sparse_lin_sys(N):
    # Form matrix A
    A = scipy.sparse.diags([-1, 2, -1], [-1, 0, 1], shape=(N, N))
    
    # Form vector b
    h = 1/(N+1)
    b = np.zeros((N,))
    for i in range(N):
        b[i] = h**2 * f( (i+1)*h )

    return A, b 

In [90]:
# Testing out functionality of the function
N_test = 10
A_test, b_test = form_sparse_lin_sys(N_test)
A_test

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements (3 diagonals) in DIAgonal format>

In [91]:
# Printing out non-sparse version of the sparse matrix generated, just for testing
# A.toarray()

In [92]:
b_test

array([ 0.17639375,  0.29678373,  0.32294698,  0.24657684,  0.09192029,
       -0.09192029, -0.24657684, -0.32294698, -0.29678373, -0.17639375])

## Solving the system with SciPy

In [94]:
x_star = scipy.sparse.linalg.spsolve(A_test, b_test)
x_star

array([ 0.55558324,  0.93477272,  1.01717847,  0.77663724,  0.28951918,
       -0.28951918, -0.77663724, -1.01717847, -0.93477272, -0.55558324])

## Check if matrix is strictly diagonally dominant

In [95]:
def strictDiagDom(A):
  # Check if input matrix A is strictly diagonally dominant
    if not scipy.sparse.isspmatrix(A):
        print("Input matrix is not a sparse matrix")
        return False
    
    diag = np.abs(A.diagonal())
    off_diag = np.abs(A.sum(axis=1) - diag)
    isDom = np.all(diag > off_diag)
    
    if isDom:
        print("A is strictly diagonally dominant")
    else:
        print("A is not strictly diagonally dominant")
        
    return isDom

## Jacobi Method for solving Ax = b

In [168]:
def jacobi(A, b, n, max_iter):
    # Define tolerance for error
    tol = 1e-10

    # Check if input matrix A is strictly diagonally dominant
    if not strictDiagDom(A):
        print('A is not strictly diagonally dominant, may not converge')

    # Convert A to CSR format
    A = scipy.sparse.csr_matrix(A)
    x = np.zeros_like(n)
    x_star = scipy.sparse.linalg.spsolve(A, b)

    for i in range(max_iter):
        # Compute residual
        r = b - A.dot(x)
        D = A.diagonal()

        # Compute the Jacobi update
        delta = np.divide(r, D)
        x_new = x + delta

        # Check for convergence
        if np.linalg.norm(delta) < tol:
            break

        x = x_new

    # Compute iteration matrix and spectral radius
    M = scipy.sparse.diags(1/D, 0).dot(A)
    spectral_radius = np.abs(scipy.sparse.linalg.eigs(M, k=1, return_eigenvectors=False))[0]

    # Compute the error
    err_st1 = abs(x_star - x)
    err = scipy.linalg.norm(err_st1)

    return x, spectral_radius, err

In [169]:
jac_estimate, jac_specRad, jac_error = jacobi(A_test, b_test, N_test, 1000)
jac_estimate

array([ 0.55558324,  0.93477272,  1.01717847,  0.77663724,  0.28951918,
       -0.28951918, -0.77663724, -1.01717847, -0.93477272, -0.55558324])

In [170]:
x_star

array([ 0.55558324,  0.93477272,  1.01717847,  0.77663724,  0.28951918,
       -0.28951918, -0.77663724, -1.01717847, -0.93477272, -0.55558324])

## Gauss-Seidel Method for Solving Ax = b

In [177]:
def gaussSeidel(A, b, n, max_iter):
    # Define tolerance for error
    tol = 1e-10

    # Check if input matrix A is strictly diagonally dominant
    if not strictDiagDom(A):
        print('A is not strictly diagonally dominant, may not converge')
    
    # Convert A to CSR format
    A = scipy.sparse.csr_matrix(A)
    x = np.zeros(n)
    x_star = scipy.sparse.linalg.spsolve(A, b)

    for i in range(max_iter):
        for j in range(n):
            # Compute the residual for the jth equation
            r_j = b[j] - A[j,:].dot(x)
            D_jj = A[j,j]

            # Compute the Gauss-Seidel update for the jth variable
            x[j] += r_j / D_jj

        # Check for convergence
        if np.linalg.norm(r_j) < tol:
            break

    # Compute iteration matrix and spectral radius
    M = scipy.sparse.tril(A, k=-1).dot(scipy.sparse.linalg.spsolve(scipy.sparse.triu(A, k=0), scipy.sparse.eye(n)))
    spectral_radius = np.abs(scipy.sparse.linalg.eigs(M, k=1, return_eigenvectors=False))[0]

    # Compute the error
    err_st1 = abs(x_star - x)
    err = scipy.linalg.norm(err_st1)

    return x, spectral_radius, err

In [172]:
gauss_estimate, gauss_specRad, gauss_error = gaussSeidel(A_test, b_test, N_test, 1000)
gauss_estimate

array([ 0.55558324,  0.93477272,  1.01717847,  0.77663725,  0.28951918,
       -0.28951918, -0.77663724, -1.01717847, -0.93477272, -0.55558324])

In [173]:
x_star

array([ 0.55558324,  0.93477272,  1.01717847,  0.77663724,  0.28951918,
       -0.28951918, -0.77663724, -1.01717847, -0.93477272, -0.55558324])

## Testing out with different matrices

In [134]:
N_sizes = [10, 20, 40, 80, 160]
for i in N_sizes:
  A_curr, b_curr = form_sparse_lin_sys(i)
  x_star_curr = scipy.sparse.linalg.spsolve(A_curr, b_curr)
  jac_estimate, jac_specRad, jac_error = jacobi(A_curr, b_curr, i, 1000)
  gauss_estimate, gauss_specRad, gauss_error= gaussSeidel(A_curr, b_curr, i, 1000)

  print('Current Matrix Size N:', i) 
  print(jac_estimate)
  print(gauss_estimate)
  print('Jacobi Method Spectral Radius:', jac_specRad)
  print('Gauss-Seidel Spectral Radius:', gauss_specRad)
  print('')

Current Matrix Size N: 10
[ 0.55558324  0.93477272  1.01717847  0.77663724  0.28951918 -0.28951918
 -0.77663724 -1.01717847 -0.93477272 -0.55558324]
[ 0.55558324  0.93477272  1.01717847  0.77663725  0.28951918 -0.28951918
 -0.77663724 -1.01717847 -0.93477272 -0.55558324]
Jacobi Method Spectral Radius: 1.9594929736144946
Gauss-Seidel Spectral Radius: 0.9206267664155896

Current Matrix Size N: 20
[ 0.29696393  0.56754131  0.78769016  0.93784928  1.00467637  0.98223356
  0.87251499  0.68526963  0.43713506  0.15015912 -0.15015912 -0.43713506
 -0.68526963 -0.87251499 -0.98223356 -1.00467637 -0.93784928 -0.78769016
 -0.56754131 -0.29696393]
[ 0.29696393  0.56754132  0.78769017  0.93784929  1.00467638  0.98223358
  0.872515    0.68526965  0.43713508  0.15015913 -0.1501591  -0.43713504
 -0.68526962 -0.87251498 -0.98223355 -1.00467636 -0.93784927 -0.78769015
 -0.56754131 -0.29696393]
Jacobi Method Spectral Radius: 1.9888308262251309
Gauss-Seidel Spectral Radius: 0.9777864028930712

Current Matr

## Check iterations

In [241]:
N_iter = 100
A_iter, b_iter = form_sparse_lin_sys(N_iter)
x_star_iter = scipy.sparse.linalg.spsolve(A_iter, b_iter)

In [239]:
jac_errors_iters = []
gauss_errors_iters = []

In [240]:
jac_specrad_iters = []
gauss_specrad_iters = []

In [253]:
current_iter = 100
# Jacobi Testing
jac_estimate, jac_specRad, jac_error = jacobi(A_iter, b_iter, N_iter, current_iter)
jac_errors_iters.append(jac_error)
jac_specrad_iters.append(jac_specRad)
# Gauss-Seidel Testing
gauss_estimate, gauss_specRad, gauss_error = gaussSeidel(A_iter, b_iter, N_iter, current_iter)
gauss_errors_iters.append(gauss_error)
gauss_specrad_iters.append(gauss_specRad)

In [254]:
jac_errors_iters

[4.483397476812058,
 4.032078494457569,
 3.5836938880328084,
 3.18517159345417,
 2.8309667055063206,
 2.516150936469359,
 2.2363440455804744,
 1.9876528938366993,
 1.7666172761677774,
 1.570161777306207,
 1.3955529814932275]

In [255]:
gauss_errors_iters

[4.433605189224998,
 3.6032739754310357,
 2.8592588938131285,
 2.267929417788103,
 1.7990152941449997,
 1.4279967321726026,
 1.1351204094202614,
 0.9045264266480328,
 0.7235124330010749,
 0.5819200962597871,
 0.4716255825482128]

In [256]:
jac_specrad_iters

[1.9970658011837406,
 1.9970658011837443,
 1.9970658011837377,
 1.9970658011837443,
 1.997065801183743,
 1.9970658011837434,
 1.99706580118374,
 1.9970658011837386,
 1.9970658011837428,
 1.9970658011837392,
 1.9970658011837361]

In [257]:
gauss_specrad_iters

[0.9941402118901742,
 0.9941402118901723,
 0.9941402118901739,
 0.9941402118901734,
 0.9941402118901738,
 0.9941402118901761,
 0.9941402118901793,
 0.9941402118901742,
 0.9941402118901743,
 0.9941402118901747,
 0.9941402118901744]

In [217]:
jac_nk_errors = []
gauss_nk_errors = []

In [218]:
for curr in range(len(gauss_errors_iters)):
    jac_curr_nk = jac_errors_iters[curr] / jac_errors_iters[0]
    jac_nk_errors.append(jac_curr_nk)
    gauss_curr_nk = gauss_errors_iters[curr] / gauss_errors_iters[0]
    gauss_nk_errors.append(gauss_curr_nk)

In [260]:
# Create error table with sizes
# Specify the Column Names while initializing the Table
myTable_sr = PrettyTable(["Iteration", "Jacobi Spectral Radius", "Gauss-Seidel Spectral Radius"])
iterNum_sr = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# Add rows
for i in range(0,10):
  myTable_sr.add_row([iterNum_sr[i], jac_specrad_iters[i], gauss_specrad_iters[i]])

print(myTable_sr)

+-----------+------------------------+------------------------------+
| Iteration | Jacobi Spectral Radius | Gauss-Seidel Spectral Radius |
+-----------+------------------------+------------------------------+
|     1     |   1.9970658011837406   |      0.9941402118901742      |
|     10    |   1.9970658011837443   |      0.9941402118901723      |
|     20    |   1.9970658011837377   |      0.9941402118901739      |
|     30    |   1.9970658011837443   |      0.9941402118901734      |
|     40    |   1.997065801183743    |      0.9941402118901738      |
|     50    |   1.9970658011837434   |      0.9941402118901761      |
|     60    |    1.99706580118374    |      0.9941402118901793      |
|     70    |   1.9970658011837386   |      0.9941402118901742      |
|     80    |   1.9970658011837428   |      0.9941402118901743      |
|     90    |   1.9970658011837392   |      0.9941402118901747      |
+-----------+------------------------+------------------------------+


In [261]:
# Create error table with sizes
# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Iteration", "Jacobi Absolute Error", "Gauss-Seidel Absolute Error", "Jacobi Relative Error", "Gauss-Seidel Relative Error"])
iterNum = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# Add rows
for i in range(0,10):
  myTable.add_row([iterNum[i], jac_errors_iters[i], gauss_errors_iters[i], jac_nk_errors[i], gauss_nk_errors[i]])

print(myTable)

+-----------+-----------------------+-----------------------------+-----------------------+-----------------------------+
| Iteration | Jacobi Absolute Error | Gauss-Seidel Absolute Error | Jacobi Relative Error | Gauss-Seidel Relative Error |
+-----------+-----------------------+-----------------------------+-----------------------+-----------------------------+
|     1     |   4.483397476812058   |      4.433605189224998      |          1.0          |             1.0             |
|     10    |   4.032078494457569   |      3.6032739754310357     |   0.8993354961970935  |      0.8127187292608015     |
|     20    |   3.5836938880328084  |      2.8592588938131285     |   0.799325490672536   |      0.6449060689395602     |
|     30    |    3.18517159345417   |      2.267929417788103      |   0.7104370312754428  |      0.5115316589983829     |
|     40    |   2.8309667055063206  |      1.7990152941449997     |   0.6314333538679004  |      0.4057680414388613     |
|     50    |   2.516150